<a href="https://colab.research.google.com/github/jaideep99/GFA-Predictor/blob/main/PGFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
import re
import pickle
import math
import tensorflow as tf

In [ ]:
param = pd.read_csv("/content/drive/MyDrive/Mendeley Dataset/New_Folder/final_dataset_parameters.csv")
param.head()

In [ ]:
# melting-point, enthalphy, electro-negativity, atomic-radius of elements
param.set_index('Unnamed: 0', drop=True, inplace=True)
pm = param.to_dict(orient="index")

In [12]:
# seperates big-small atoms
def diff(x):

  #making ranges for each atom
  ranges = {}
  for i in x:
    ranges[i] = 0.88*pm[i]['ar']
  
  #compiling scoring matrix
  score = {}
  
  for i in x:
    dct = {}
    for j in x:
      if pm[i]['ar']<ranges[j]:
        dct[j] = -1
      elif pm[i]['ar']>pm[j]['ar']:
        dct[j] = 1
      else :
        dct[j] = 0
          
    score[i] = dct
    
  b = []
  sm = []
  print()
  #separating into big and small based on scoring matrix
  for i in score:
    sum = 0
    for j in score[i]:
      sum = sum+score[i][j]
      
    if(sum>0):
      b.append(i)
      
    else : 
      sm.append(i)
      
      
  return b,sm

In [13]:
#finds the paramater deltaE
def electro(all,x):
  k =0 #summation of product of composition and electro negativity
  p = 0 #summation of compositions

  for i in all:
    if i in x:
      p = p+all[i]
  for i in all:
    if i in x:
      k = k+all[i]*pm[i]['en']
  
  return k/p

In [14]:
#finds the paramater deltaD
def comps(all,x):
  k =0 #summation of product of composition and atomic radii
  p = 0 #summation of compositions

  for i in all:
    if i in x:
      p = p+all[i]
  for i in all:
    if i in x:
      k = k+all[i]*pm[i]['ar']
  
  return k/p

In [15]:
def prepare_params(x):
  #Seperating atoms from composition
  s = re.sub(r'[^\w\s]','',x)
  s = re.sub('\d',' ',s)
  els = np.array([i for i in s.split(' ') if i in pm])
  print('\nElements in BMG are : ', els)
  
  comp = re.findall(r"[-+]?\d*\.\d+|\d+",x)
  comp = [float(i) for i in comp]
  print('Compositions: ',comp)
  
  dcomp = dict(zip(els,comp))
  smix = 0
  hmix = 0

  for i in dcomp:
    smix = smix + (dcomp[i]/100)*(math.log((dcomp[i]/100)))
    hmix = hmix + (dcomp[i]/100)*pm[i]['enthalphy']
  smix = -1*smix

  big,small = diff(els)

  maxrad = 0.5137797338363198
  maxen = 0.1441441441441441
  rad = (comps(dcomp,big)-comps(dcomp,small))/(comps(dcomp,big))
  rad = int((rad/maxrad)*100)
  en = (electro(dcomp,big)-electro(dcomp,small))/(electro(dcomp,big)+electro(dcomp,small))
  en = int((en/maxen)*100)

  return [hmix,smix,rad,en]

In [20]:
# Loads ANN model and applies it to predict GFA of given composition
def predictGfaANN(bmg):
  model = tf.keras.models.load_model('drop_model_final.h5')
  inp = np.array([bmg])
  pred = model.predict(inp)
  return pred[0][0]

In [31]:
# Loads RF model and applies it to predict GFA of given composition
def predictGfaRF(bmg):
  rf_model = pickle.load(open('random_forest.sav', 'rb'))
  inp = np.array([bmg])
  pred = rf_model.predict(inp)
  return pred[0]

In [28]:
# Applies pre-computed standard scaler on params/input
def scaleParams(bmg):
  scaler = pickle.load(open('st_scaler.sav', 'rb'))
  bmg = np.array([bmg])
  params_scaled = scaler.transform(bmg)
  return params_scaled[0]

In [32]:
bmg = input("Enter BMG elements and compositions : ")

params = prepare_params(bmg)


print("Computed Parameters are : ",params)

params = scaleParams(params)
print("Computed Scaled Parameters are : ",params)


GFA_ANN = predictGfaANN(params)
GFA_RF = predictGfaRF(params)

print("\nPredicted Dmax ANN: ",GFA_ANN)
print("\nPredicted Dmax ANN: ",GFA_RF)

Enter BMG elements and compositions : Pd73.5Cu10Si16.5

Elements in BMG are :  ['Pd' 'Cu' 'Si']
Compositions:  [73.5, 10.0, 16.5]

Computed Parameters are :  [21.353, 0.7538524402682971, 49, 50]
Computed Scaled Parameters are :  [ 1.0567944  -1.51735497 -1.15588644  2.3168874 ]

Predicted Dmax ANN:  1.8465838

Predicted Dmax ANN:  2.28125
